In [ ]:
#selecionar variaveis das credenciais
%run ./create_conn

In [ ]:
#instalacao connector
%pip install mysql-connector-python

Python interpreter will be restarted.
Python interpreter will be restarted.


In [ ]:
# Conectar ao banco de dados MySQL (RDS)
import mysql.connector
from pyspark.sql import SparkSession
import pandas as pd

def create_connetion():
    conn = mysql.connector.connect(
        host=db_host,
        user=db_user,
        password=db_password,
        database=db_database
    )

    return conn


In [ ]:

# Conectar ao banco de dados MySQL (RDS)
conn = create_connetion()

def select_table(query):
    cursor = conn.cursor()
    cursor.execute(query)
    # Obter os nomes das colunas
    col_names = [desc[0] for desc in cursor.description]
    # Pegar os resultados da consulta
    result = cursor.fetchall()
    # Criar um SparkSession
    spark = SparkSession.builder.getOrCreate()
    # Transformar os resultados em RDD
    rdd = spark.sparkContext.parallelize(result)
    # Converter o RDD em um Spark DataFrame
    df = spark.createDataFrame(rdd, schema=col_names)
    cursor.close()
    return df


# Fase 1

### 1. Top 10 Endereços (carteiras) com maior volume de transações enviadas

In [ ]:
display(select_table('''
SELECT 
    COUNT(*) AS COUNT,
    AddressOrigin AS ADDRESS_ORIGIN, 
    SUM(CAST(REPLACE(TotalSent, ',', '') AS DOUBLE)) AS TOTAL_SENT
FROM 
    db_hiring_test.raw_transactions_table
GROUP BY 
    ADDRESS_ORIGIN
ORDER BY 
    COUNT DESC
LIMIT 
    10
'''))


COUNT,ADDRESS_ORIGIN,TOTAL_SENT
90,A-99,4.7166372E7
85,A-83,4.7298518E7
85,A-72,4.0498806E7
81,A-78,3.6950241E7
81,A-25,4.5302857E7
80,A-48,3.5950322E7
79,A-62,3.1364981E7
78,A-32,3.1185309E7
77,A-54,3.4919296E7
76,A-50,3.5920647E7


Databricks visualization. Run in Databricks to view.

### 2. Top 10 dias do mês com maiores volumes de transações realizadas

In [ ]:
display(select_table('''
SELECT 
       COUNT(*) AS COUNT,
       DAY(STR_TO_DATE(SentDate,'%Y-%m-%d %H:%i:%s'))  AS DAY_OF_MONTH,
       SUM(CAST(REPLACE(TotalSent, ',', '') AS DOUBLE)) AS TOTAL_SENT
FROM 
       db_hiring_test.raw_transactions_table
WHERE 
       Status = 'Confirmed'
GROUP BY 
       DAY_OF_MONTH
ORDER BY 
       COUNT DESC
LIMIT 
       10
'''))

COUNT,DAY_OF_MONTH,TOTAL_SENT
286,7,1.5475931E8
279,2,1.28156372E8
274,15,1.50092292E8
262,27,1.29371428E8
258,12,1.35455458E8
236,1,1.14533986E8
229,14,1.15944578E8
226,13,1.21662984E8
226,20,1.06683771E8
220,19,1.21721712E8


Databricks visualization. Run in Databricks to view.

### 3. Dias da semana com maiores volumes de transações realizadas

In [ ]:
display(select_table('''
SELECT 
       COUNT(*) AS COUNT,
       DAYNAME(STR_TO_DATE(SentDate,'%Y-%m-%d %H:%i:%s'))  AS DAY_OF_MONTH
FROM 
       db_hiring_test.raw_transactions_table
WHERE 
       Status = 'Confirmed'
GROUP BY 
       DAY_OF_MONTH
ORDER BY 
       COUNT DESC
'''))

COUNT,DAY_OF_MONTH
970,Saturday
927,Friday
921,Wednesday
898,Thursday
865,Sunday
840,Tuesday
694,Monday


Databricks visualization. Run in Databricks to view.

### 4. Transações que possuem condições atípicas

In [ ]:
display(select_table('''
SELECT 
   * 
FROM 
   db_hiring_test.raw_transactions_table
WHERE 
   AddressOrigin IS NULL 
   OR AddressOrigin = ''
   OR AddressDestination IS NULL 
   OR AddressDestination = ''
   OR TotalSent IS NULL 
   OR TotalSent = ''
   OR Status IS NULL 
   OR Status = ''
   OR Status <> 'Confirmed'
'''))

IdTransaction,AddressOrigin,AddressDestination,TotalSent,Status,SentDate,ImportDate
ID2732,A-3,A-50,"998,501.00",Denied,2021-01-20 16:29:47,2021-01-31 23:59:59
ID266,A-50,A-84,"100,156.00",Denied,2021-01-12 21:07:28,2021-01-31 23:59:59
ID1216,A-88,A-67,"502,360.00",Pending,2021-01-24 21:45:26,2021-01-31 23:59:59
ID965,A-74,A-45,"426,923.00",Denied,2021-01-24 19:19:47,2021-01-31 23:59:59
ID2837,A-74,A-100,"554,294.00",Denied,2021-01-20 06:18:18,2021-01-31 23:59:59
ID2832,A-7,A-16,"119,209.00",Pending,2021-01-28 21:45:26,2021-01-31 23:59:59
ID1158,A-22,A-76,"5,989.00",Denied,2021-01-07 03:07:57,2021-01-31 23:59:59
ID2523,A-94,A-35,"773,785.00",Pending,2021-01-23 19:33:41,2021-01-31 23:59:59
ID2593,A-88,A-71,"804,143.00",Denied,2021-01-09 11:48:41,2021-01-31 23:59:59
ID2249,A-58,A-33,"940,414.00",Pending,2021-01-10 10:39:40,2021-01-31 23:59:59


### 5. Carteira com o maior saldo final

In [ ]:
display(select_table('''
SELECT 
    AddressDestination AS ADDRESS_DESTINATION, 
    SUM(CAST(REPLACE(TotalSent, ',', '') AS DOUBLE)) AS TOTAL_SENT
FROM 
    db_hiring_test.raw_transactions_table
GROUP BY 
    ADDRESS_DESTINATION
ORDER BY 
    TOTAL_SENT DESC
LIMIT 
    1
'''))

ADDRESS_DESTINATION,TOTAL_SENT
A-12,4.6125129E7


Databricks visualization. Run in Databricks to view.

# Fase 2

### Maior pagamento de taxas em janeiro e fevereiro de 2021

In [ ]:
display(select_table('''
WITH RTT_TRANSFORM AS (
    SELECT 
        AddressOrigin,
        CAST(REPLACE(TotalSent, ',', '') AS DOUBLE) AS Total_Sent,
        YEAR(STR_TO_DATE(SentDate,'%Y-%m-%d %H:%i:%s')) AS SendYear,
        MONTH(STR_TO_DATE(SentDate,'%Y-%m-%d %H:%i:%s')) AS SendMonth
    FROM 
        db_hiring_test.raw_transactions_table
    WHERE Status = 'Confirmed'
),
RTF_TRANSFORM AS (
    SELECT 
        `range-start` AS R_START,
        `range-end` AS R_END,
        CAST(`fee-percentage` AS DOUBLE) AS F_PERCENTAGE
    FROM 
        db_hiring_test.raw_transactions_fee
),
RTT_RTF_MATCH AS (
    SELECT 
        rtt.*, 
        rtf.F_PERCENTAGE
    FROM 
        RTT_TRANSFORM rtt
    JOIN 
        RTF_TRANSFORM rtf 
    ON 
        rtt.Total_Sent BETWEEN rtf.R_START AND rtf.R_END
)

SELECT 
    AddressOrigin AS ADDRESS_ORIGIN,
    SUM(Total_Sent) AS SUM_TOTAL_SENT,
    SendYear SEND_YEAR,
    SendMonth SEND_MONTH,
    SUM(F_PERCENTAGE) AS SUM_PERCENTAGE

FROM 
    RTT_RTF_MATCH
WHERE 
    SendYear = 2021 AND SendMonth = 1
GROUP BY 
    ADDRESS_ORIGIN
ORDER BY 
    SUM_PERCENTAGE DESC
LIMIT 
    1
'''))


ADDRESS_ORIGIN,SUM_TOTAL_SENT,SEND_YEAR,SEND_MONTH,SUM_PERCENTAGE
A-91,3048822.0,2021,2,34.0


Databricks visualization. Run in Databricks to view.

In [ ]:
display(select_table('''
WITH RTT_TRANSFORM AS (
    SELECT 
        AddressOrigin,
        CAST(REPLACE(TotalSent, ',', '') AS DOUBLE) AS Total_Sent,
        YEAR(STR_TO_DATE(SentDate,'%Y-%m-%d %H:%i:%s')) AS SendYear,
        MONTH(STR_TO_DATE(SentDate,'%Y-%m-%d %H:%i:%s')) AS SendMonth
    FROM 
        db_hiring_test.raw_transactions_table
    WHERE Status = 'Confirmed'
),
RTF_TRANSFORM AS (
    SELECT 
        `range-start` AS R_START,
        `range-end` AS R_END,
        CAST(`fee-percentage` AS DOUBLE) AS F_PERCENTAGE
    FROM 
        db_hiring_test.raw_transactions_fee
),
RTT_RTF_MATCH AS (
    SELECT 
        rtt.*, 
        rtf.F_PERCENTAGE
    FROM 
        RTT_TRANSFORM rtt
    JOIN 
        RTF_TRANSFORM rtf 
    ON 
        rtt.Total_Sent BETWEEN rtf.R_START AND rtf.R_END
)

SELECT 
    AddressOrigin AS ADDRESS_ORIGIN,
    SUM(Total_Sent) AS SUM_TOTAL_SENT,
    SendYear SEND_YEAR,
    SendMonth SEND_MONTH,
    SUM(F_PERCENTAGE) AS SUM_PERCENTAGE

FROM 
    RTT_RTF_MATCH
WHERE 
    SendYear = 2021 AND SendMonth = 2
GROUP BY 
    ADDRESS_ORIGIN
ORDER BY 
    SUM_PERCENTAGE DESC
LIMIT 
    1
'''))


ADDRESS_ORIGIN,SUM_TOTAL_SENT,SEND_YEAR,SEND_MONTH,SUM_PERCENTAGE
A-91,3048822.0,2021,2,34.0


Databricks visualization. Run in Databricks to view.

### 3. ID da transação com a maior taxa paga

In [ ]:
display(select_table('''
WITH RTT_TRANSFORM AS (
    SELECT 
        IdTransaction,
        CAST(REPLACE(TotalSent, ',', '') AS DOUBLE) AS Total_Sent
    FROM 
        db_hiring_test.raw_transactions_table
    WHERE Status = 'Confirmed'
),
RTF_TRANSFORM AS (
    SELECT 
        `range-start` AS R_START,
        `range-end` AS R_END,
        CAST(`fee-percentage` AS DOUBLE) AS F_PERCENTAGE
    FROM 
        db_hiring_test.raw_transactions_fee
),
RTT_RTF_MATCH AS (
    SELECT 
        rtt.*, 
        rtf.F_PERCENTAGE
    FROM 
        RTT_TRANSFORM rtt
    JOIN 
        RTF_TRANSFORM rtf 
    ON 
        rtt.Total_Sent BETWEEN rtf.R_START AND rtf.R_END
)

SELECT 
    IdTransaction AS ID_TRANSACTION,
    Total_Sent AS TOTAL_SENT,
    F_PERCENTAGE AS PERCENTAGE

FROM 
    RTT_RTF_MATCH
ORDER BY 
    PERCENTAGE DESC
LIMIT 
    1
'''))


ID_TRANSACTION,TOTAL_SENT,PERCENTAGE
ID1092,57493.0,10.0


Databricks visualization. Run in Databricks to view.

### 4. Média de taxa paga considerando todas as transações realizadas

In [ ]:
display(select_table('''
WITH RTT_TRANSFORM AS (
    SELECT 
        IdTransaction,
        CAST(REPLACE(TotalSent, ',', '') AS DOUBLE) AS Total_Sent
    FROM 
        db_hiring_test.raw_transactions_table
    WHERE Status = 'Confirmed'
),
RTF_TRANSFORM AS (
    SELECT 
        `range-start` AS R_START,
        `range-end` AS R_END,
        CAST(`fee-percentage` AS DOUBLE) AS F_PERCENTAGE
    FROM 
        db_hiring_test.raw_transactions_fee
),
RTT_RTF_MATCH AS (
    SELECT 
        rtt.*, 
        rtf.F_PERCENTAGE
    FROM 
        RTT_TRANSFORM rtt
    JOIN 
        RTF_TRANSFORM rtf 
    ON 
        rtt.Total_Sent BETWEEN rtf.R_START AND rtf.R_END
)

SELECT 
    ROUND(AVG(F_PERCENTAGE),2) AS MEAN_PERCENTAGE
FROM 
    RTT_RTF_MATCH
'''))


MEAN_PERCENTAGE
5.87


Databricks visualization. Run in Databricks to view.